# Общие замечания по первому ДЗ.

In [3]:
import numpy as np

1.1.6 Даны: исходный массив $A$ и $B$ (одномерные). Как вычислить выражения $-((B-А)*(B/2))$ "in place" т.е. используя для промежуточных результатов и конечного резульатата ихсодный массив $A$?

К сожалению, большинство студентов не справились с этим заданием. Непонимание механизма работы `ndarray` в будущем доставит вам немало хлопот, поэтому давайте подробнее окунемся в эту тему.  
В основе этой заметки - ответ на [stackoverflow](https://stackoverflow.com/questions/35910577/why-does-python-numpys-mutate-the-original-array#:~:text=Numpy%20arrays%20are%20mutable%20objects,the%20operation%20is%20a%20%3D%20np).

Многие из вас должно быть слышали, что в python объекты бывают двух видов: mutable и immutable. Разница состоит в том, что в mutable-объектах значения полей можно изменять после их создания, в immutable - нет.  
Преимуществом mutable объектов является возможность хранить различные состояния, которые меняются в ходе выполнения программы. Такой подход  здорово упрощает реализацию бизнес логики приложения.  
Immutable объекты нельзя изменить после их создания, можно лишь создавать новые объекты на их основе. Такой подход имеет своё преимущество - можно писать код в функциональной парадигме. Это упрощает разработку более производительных многопоточных приложений.  
В библиотеке numpy объекты типа ndarray по умолчанию создаются mutable. Создадим массив и изменим его элемент.

In [9]:
a = np.arange(10)
a[0] = 10
print(a)

[10  1  2  3  4  5  6  7  8  9]


Однако созданный экземпляр ndarray можно сделать immutable установкой флага `writable = False`  

Попытка изменить значение элемента теперь вызовет ошибку.

In [10]:
a = np.arange(10)
a.flags.writeable = False
a[0] = 10
print(a)

ValueError: assignment destination is read-only

Давайте рассмотрим задание.

**Дано:**  
Исходные массивы $A$ и $B$ (одномерные).  
**Требуется:**  
Вычислить выражения $-((B-А)*(B/2))$ "in place" т.е. используя для промежуточных результатов и конечного результата исходный массив $A$

In [31]:
A = np.ones(4)
B = np.ones(4) * 2
print(A, B)

[1. 1. 1. 1.] [2. 2. 2. 2.]


Посмотрим на адрес объекта `A` в памяти (идентификатор `id`): 

In [32]:
id(A)

139704608056944

Вот пример решения, которое позволяет получить правильный ответ.

In [33]:
A = -A
A += B
B = B/2
A *= B
A = -A
print(A)

[-1. -1. -1. -1.]


Однако остался ли объект `A` на прежнем месте? 

In [34]:
id(A)

139704608057664

Нет.  
Дело в том, что в python оператор `+` (сложение) вызывает метод `__add__` левого операнда или в некоторых редких случаях метод `__radd__` правого операнда.  
Оператор `+=` (сложение с присвоением) работает по-другому, `A += B` не всегда эквивалентно `A = A + B`.  
Если для левого операнда определен метод `__iadd__`, то он и вызывается для оператора `+=`. В этом случае осуществляется in-place add, т.е. результат сложения будет записан в этот же объект, адрес объекта в памяти не изменяется.  
Если же метод `__iadd__` не определен, то осуществляется обычное сложение с присвоением: `A += B` которое эквивалентно `A = A + B`. В результате сложения `A + B` создается новый объект, ссылка на который который записывается в переменную `A`.  
Этот механизм работает так же для вычитания, умножения и деления.

Для того, чтобы решить задачу с вычислениями `in-place`, немного упростим выражение: $-((B-А)*(B/2)) => (A-B)*B/2$ 

In [43]:
A = np.ones(4)
B = np.ones(4) * 2
print(id(A))

139704608714064


In [44]:
A -= B
A *= B
A /= 2
print(A)

[-1. -1. -1. -1.]


In [45]:
id(A)

139704608714064

Как видите, адрес объекта не изменился, т.е. все вычисления осуществлялись `in-place`.  
Напоследок вспомним, что в `numpy` есть функции, реализующие базовую арифметику.  
Выражение `a = a + b` эквивалентно вызову функции `a = np.add(a, b)`. Выражение `a += b` эквивалентно `a = np.add(a, b, out=a)`

Более подробно про выделение памяти под объекты в python можно почитать в статье www.math.buffalo.edu/mutable_vs_immutable

1.6.2 Найти все простые числа в пределах ста. (Для решения предлагается использовать Решето Эратосфена) Использовать не более 1 цикла (желательно).

Большинство студентов просто скопировали Вариант 1 из [wiki](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B5%D1%88%D0%B5%D1%82%D0%BE_%D0%AD%D1%80%D0%B0%D1%82%D0%BE%D1%81%D1%84%D0%B5%D0%BD%D0%B0#%D0%92%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82_%E2%84%96_1): 

In [1]:
def eratosthenes(n):     # n - число, до которого хотим найти простые числа 
    sieve = list(range(n + 1))
    sieve[1] = 0    # без этой строки итоговый список будет содержать единицу
    for i in sieve:
        if i > 1:
            for j in range(i + i, len(sieve), i):
                sieve[j] = 0
    return sieve

А хотелось бы видеть что-то такое:

In [3]:
n = 100
s = np.arange(n+1)
for i in range(2, len(s)):
    if s[i]!=0:
        s[range(2*i, len(s), i)] = 0
s[s!=0]

array([ 1,  2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53,
       59, 61, 67, 71, 73, 79, 83, 89, 97])

Вот вариант решения от Буховцевой Кристины. Недостаток - используется `list`, а хотелось бы оставаться на массивах `numpy`:

In [5]:
arr = np.array(range(3,100,2))
for j in range(0, int(round(np.sqrt(10)))):
    arr[(arr!=arr[j])&(arr%arr[j]==0)]=0
    arr=arr[arr!=0]
arr=[2]+list(arr)
arr

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

Отличный вариант от Себякина Андрея:

In [4]:
def sieve(n):
    flags = np.ones(n, dtype=bool)
    flags[0] = flags[1] = False
    for i in range(2, n):
        if flags[i]:
            flags[i*i::i] = False
    return np.flatnonzero(flags)

print(sieve(100))

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]
